Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


CPU

In [ ]:
# !pip install "transformers[flax]" flax "jax[cpu]" sentencepiece joblib tqdm -q

GPU

In [ ]:
!pip install jax[cuda12] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 16.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-cuda-nvcc-cu12
    Found existing installation: nvidia-cuda-nvcc-cu12 12.5.82
    Uninstalling nvidia-cuda-nvcc-cu12-12.5.82:
      Successfully uninstalled nvidia-cuda-nvcc-cu12-12.5.82


In [ ]:
# ============================================================
# CELL 3 — Path configuration  ✏️ EDIT HERE
# ============================================================
import os

DRIVE_ROOT     = "/content/drive/MyDrive/BTech-Final-Project"
DATA_DIR       = os.path.join(DRIVE_ROOT, "data")

# MIMIC-IV note files (discharge.csv.gz + radiology.csv.gz live here)
MIMIC_NOTE_DIR = os.path.join(DATA_DIR, "mimiciv", "note")
MIMIC_BHC_DIR  = MIMIC_NOTE_DIR   # same folder, no separate BHC dir

# ✏️ Paste your actual features.csv path here once you locate it:
FEATURES_CSV   = os.path.join(DATA_DIR, "processed", "ultimate_features.csv")

PROCESSED_DIR  = os.path.join(DATA_DIR, "processed")
# Output — embeddings will be saved here
EMBEDDINGS_DIR     = os.path.join(DATA_DIR, "processed", "embeddings")
EMBEDDINGS_CSV     = os.path.join(EMBEDDINGS_DIR, "embeddings.csv")
EMBEDDING_INFO_PKL = os.path.join(EMBEDDINGS_DIR, "embedding_info.pkl")

RANDOM_STATE = 42
os.makedirs(EMBEDDINGS_DIR, exist_ok=True)

# ── Path sanity-check — run this cell before anything else ──
def check_paths():
    print("─" * 55)
    print("PATH CHECK")
    print("─" * 55)
    all_ok = True
    checks = {
        "MIMIC note dir" : MIMIC_NOTE_DIR,
        "features.csv"   : FEATURES_CSV,
        "embeddings dir" : EMBEDDINGS_DIR,
    }
    for label, path in checks.items():
        ok = os.path.exists(path)
        print(f"  {'✅' if ok else '❌  MISSING'}  {label}\n         {path}")
        if not ok: all_ok = False

    for fn in ["discharge.csv.gz", "radiology.csv.gz"]:
        fp = os.path.join(MIMIC_NOTE_DIR, fn)
        ok = os.path.exists(fp)
        print(f"  {'✅' if ok else '❌  MISSING'}  {fn}\n         {fp}")
        if not ok: all_ok = False

    print("─" * 55)
    print("✅ All good — proceed to Cell 4." if all_ok
          else "❌ Fix missing paths above before continuing.")

check_paths()

───────────────────────────────────────────────────────
PATH CHECK
───────────────────────────────────────────────────────
  ✅  MIMIC note dir
         /content/drive/MyDrive/BTech-Final-Project/data/mimiciv/note
  ✅  features.csv
         /content/drive/MyDrive/BTech-Final-Project/data/processed/ultimate_features.csv
  ✅  embeddings dir
         /content/drive/MyDrive/BTech-Final-Project/data/processed/embeddings
  ✅  discharge.csv.gz
         /content/drive/MyDrive/BTech-Final-Project/data/mimiciv/note/discharge.csv.gz
  ✅  radiology.csv.gz
         /content/drive/MyDrive/BTech-Final-Project/data/mimiciv/note/radiology.csv.gz
───────────────────────────────────────────────────────
✅ All good — proceed to Cell 4.


In [ ]:
# ============================================================
# CELL 4 — Imports & global config
# ============================================================
import gc
import logging
import re
import warnings
from typing import Dict, List

import joblib
import numpy as np
import pandas as pd
import torch
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from tqdm.auto import tqdm   # auto-picks notebook progress bar

warnings.filterwarnings("ignore")
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s  %(levelname)-8s  %(message)s",
    datefmt="%H:%M:%S",
    force=True,
)
logger = logging.getLogger(__name__)

MODEL_NAME     = "luqh/ClinicalT5-base"
EMBEDDING_DIM  = 128    # final PCA output dims
MAX_SEQ_LEN    = 512
GPU_BATCH      = 64      # lower to 2 if OOM on Colab T4
CPU_BATCH      = 8
MIN_TEXT_LEN   = 50
MAX_TEXT_CHARS = 3000

HIGH_VALUE_SECTIONS = [
    "brief hospital course", "hospital course", "discharge diagnosis",
    "discharge condition", "assessment and plan", "assessment/plan",
    "pertinent results", "discharge medications", "history of present illness",
    "past medical history", "social history", "discharge disposition",
    "major surgical", "reason for hospitalization",
]

logger.info("Imports complete. Device: %s",
            "cuda ✅" if torch.cuda.is_available() else "cpu ⚠️  (will be slow)")


09:37:13  INFO      Imports complete. Device: cuda ✅


In [ ]:
# ============================================================
# CELL 5 — Text preprocessing helpers
# ============================================================

def preprocess_note(text: str, max_chars: int = MAX_TEXT_CHARS) -> str:
    if not isinstance(text, str) or len(text.strip()) < MIN_TEXT_LEN:
        return ""
    text = re.sub(r"\[\*\*.*?\*\*\]", " ", text)   # strip MIMIC de-id tags
    text = re.sub(r"\s+", " ", text).strip()

    extracted = []
    text_lower = text.lower()
    for sec in HIGH_VALUE_SECTIONS:
        pat = re.compile(
            rf"(?:^|\n|\r){re.escape(sec)}\s*[:\-]?\s*(.*?)(?=\n[A-Z][A-Z /]+[:\-]|\Z)",
            re.IGNORECASE | re.DOTALL,
        )
        m = pat.search(text_lower)
        if m:
            s, e = m.span(1)
            extracted.append(text[s:e].strip()[:1000])

    result = " [SEP] ".join(extracted) if extracted else text
    return result[:max_chars]


def build_file_index(dirs: List[str]) -> Dict[str, str]:
    idx = {}
    for d in dirs:
        if not os.path.isdir(d):
            continue
        for root, _, files in os.walk(d):
            for f in files:
                idx.setdefault(f, os.path.join(root, f))
    return idx


logger.info("Preprocessing helpers defined.")


09:37:13  INFO      Preprocessing helpers defined.


In [ ]:
# ============================================================
# CELL 6 — Load & merge clinical notes
# ============================================================

def load_notes(cohort_hadm: set, file_index: Dict[str, str]) -> pd.DataFrame:
    frames = []

    # ── Discharge notes ──────────────────────────────────────
    for fn in ["discharge.csv.gz", "discharge.csv"]:
        if fn in file_index:
            logger.info("Loading discharge notes from:\n  %s", file_index[fn])
            try:
                df = pd.read_csv(file_index[fn], usecols=["hadm_id", "text"],
                                 nrows=1_500_000, low_memory=True)
                logger.info("  Raw rows read     : %d", len(df))
                df = df[df["hadm_id"].isin(cohort_hadm)].dropna(subset=["text"])
                logger.info("  After cohort filter: %d rows", len(df))

                tqdm.pandas(desc="  Preprocessing discharge notes")
                df["text"] = df["text"].progress_apply(preprocess_note)

                df  = df[df["text"].str.len() >= MIN_TEXT_LEN]
                agg = df.groupby("hadm_id")["text"].apply(" [DC] ".join).reset_index()
                frames.append(agg)
                logger.info("  ✅ %d admissions with discharge notes", len(agg))
            except Exception as e:
                logger.error("  ❌ Discharge load failed: %s", e)
            break

    # ── Radiology notes ──────────────────────────────────────
    for fn in ["radiology.csv.gz", "radiology.csv"]:
        if fn in file_index:
            logger.info("Loading radiology notes from:\n  %s", file_index[fn])
            try:
                df = pd.read_csv(file_index[fn], usecols=["hadm_id", "text"],
                                 nrows=800_000, low_memory=True)
                logger.info("  Raw rows read     : %d", len(df))
                df = df[df["hadm_id"].isin(cohort_hadm)].dropna(subset=["text"])
                logger.info("  After cohort filter: %d rows", len(df))

                tqdm.pandas(desc="  Preprocessing radiology notes")
                df["text"] = df["text"].progress_apply(
                    lambda t: preprocess_note(t, max_chars=800)
                )

                df  = df[df["text"].str.len() >= MIN_TEXT_LEN]
                agg = df.groupby("hadm_id")["text"].apply(" [RAD] ".join).reset_index()
                frames.append(agg)
                logger.info("  ✅ %d admissions with radiology notes", len(agg))
            except Exception as e:
                logger.error("  ❌ Radiology load failed: %s", e)
            break

    if not frames:
        logger.warning("⚠️  No note files found! Check MIMIC_NOTE_DIR path.")
        return pd.DataFrame(columns=["hadm_id", "text"])

    # ── Merge ────────────────────────────────────────────────
    notes = frames[0]
    for fr in frames[1:]:
        notes = notes.merge(fr, on="hadm_id", how="outer", suffixes=("", "_y"))
        if "text_y" in notes.columns:
            notes["text"] = notes["text"].fillna("") + " " + notes["text_y"].fillna("")
            notes = notes.drop(columns=["text_y"])

    notes["text"] = notes["text"].str.strip()
    notes = notes[notes["text"].str.len() >= MIN_TEXT_LEN].drop_duplicates("hadm_id")
    logger.info("✅ Merged notes: %d unique admissions", len(notes))
    return notes


In [ ]:
# CELL 7a — install msgpack-numpy (needed for direct Flax deserialization)
!pip install msgpack-numpy -q

In [ ]:
# ============================================================
# CELL 7 — Load ClinicalT5 (pure numpy → PyTorch, no Flax libs)
# ============================================================
# !pip install msgpack msgpack-numpy -q   ← run this once first

CONVERTED_MODEL_DIR = "/content/clinical_t5_pytorch"

def _msgpack_restore(data):
    import numpy as np
    result = {}
    def _traverse(obj, prefix=""):
        if isinstance(obj, dict):
            for k, v in obj.items():
                new_prefix = f"{prefix}/{k}" if prefix else k
                _traverse(v, new_prefix)
        elif isinstance(obj, (bytes, bytearray)):
            # msgpack_numpy encodes arrays as raw bytes with metadata
            # try to decode as float32 array
            try:
                arr = np.frombuffer(obj, dtype=np.float32)
                result[prefix] = arr
            except Exception:
                pass
        elif hasattr(obj, 'shape'):   # already a numpy array
            result[prefix] = obj
    _traverse(data)
    return result


def _flax_key_to_pytorch(key: str):
    """
    Convert a Flax parameter path like 'encoder/block/0/layer/0/SelfAttention/q/kernel'
    into a PyTorch state_dict key like 'encoder.block.0.layer.0.SelfAttention.q.weight'.
    """
    key = key.replace("/", ".")
    # Flax uses 'kernel' for weight matrices (transposed vs PyTorch)
    if key.endswith(".kernel"):
        key = key[:-7] + ".weight"
    elif key.endswith(".scale"):
        key = key[:-6] + ".weight"
    elif key.endswith(".embedding"):
        key = key[:-10] + ".weight"
    return key


def load_clinical_t5(device: torch.device):
    from transformers import AutoTokenizer, T5EncoderModel, T5Config
    import numpy as np

    logger.info("Loading ClinicalT5 (%s) ...", MODEL_NAME)
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    pt_weights  = os.path.join(CONVERTED_MODEL_DIR, "model.safetensors")
    pt_weights2 = os.path.join(CONVERTED_MODEL_DIR, "pytorch_model.bin")

    if os.path.exists(pt_weights) or os.path.exists(pt_weights2):
        logger.info("  ✅ Cached PyTorch model found — skipping conversion")
    else:
        logger.info("  No PyTorch weights found — converting from Flax msgpack ...")
        os.makedirs(CONVERTED_MODEL_DIR, exist_ok=True)

        # 1. Download raw msgpack
        from huggingface_hub import hf_hub_download
        logger.info("  Downloading flax_model.msgpack (~892 MB) ...")
        msgpack_path = hf_hub_download(repo_id=MODEL_NAME, filename="flax_model.msgpack")
        logger.info("  Saved to: %s", msgpack_path)

        # 2. Deserialize msgpack → nested dicts of numpy arrays
        logger.info("  Deserializing msgpack ...")
        import msgpack
        import msgpack_numpy
        msgpack_numpy.patch()
        with open(msgpack_path, "rb") as f:
            raw = msgpack.unpackb(f.read(), raw=False)

        # 3. Build PyTorch model with random weights, then overwrite
        logger.info("  Building T5EncoderModel skeleton ...")
        config   = T5Config.from_pretrained(MODEL_NAME)
        pt_model = T5EncoderModel(config)
        pt_sd    = pt_model.state_dict()

        # 4. Flatten flax params and map to PyTorch keys
        logger.info("  Mapping Flax params → PyTorch state_dict ...")
        flat = _msgpack_restore(raw)
        logger.info("  Flax param tensors: %d", len(flat))

        matched, skipped = 0, 0
        new_sd = {}
        for flax_key, arr in flat.items():
            pt_key = _flax_key_to_pytorch(flax_key)
            if pt_key in pt_sd:
                t = torch.from_numpy(np.array(arr, dtype=np.float32))
                # Flax kernels are (in, out); PyTorch Linear weights are (out, in)
                if t.ndim == 2 and t.shape[::-1] == tuple(pt_sd[pt_key].shape):
                    t = t.T
                if t.shape == pt_sd[pt_key].shape:
                    new_sd[pt_key] = t
                    matched += 1
                else:
                    logger.debug("  Shape mismatch %s: flax=%s pt=%s",
                                 pt_key, t.shape, pt_sd[pt_key].shape)
                    skipped += 1
            else:
                skipped += 1

        logger.info("  Matched: %d / %d params | Skipped: %d",
                    matched, len(pt_sd), skipped)

        # Fill unmatched keys with original random init (shouldn't matter for matched ones)
        for k in pt_sd:
            if k not in new_sd:
                new_sd[k] = pt_sd[k]

        pt_model.load_state_dict(new_sd, strict=False)

        # 5. Save
        logger.info("  Saving converted model to %s ...", CONVERTED_MODEL_DIR)
        pt_model.save_pretrained(CONVERTED_MODEL_DIR)
        tokenizer.save_pretrained(CONVERTED_MODEL_DIR)
        logger.info("  ✅ Conversion complete")
        del raw, flat, new_sd, pt_model
        gc.collect()

    # Load converted model
    dtype = torch.float16 if device.type == "cuda" else torch.float32
    model = T5EncoderModel.from_pretrained(
        CONVERTED_MODEL_DIR,
        torch_dtype=dtype,
        low_cpu_mem_usage=True,
    ).to(device).eval()

    n_params = sum(p.numel() for p in model.parameters()) / 1e6
    logger.info("✅ ClinicalT5 ready — %.1fM params | device=%s | dtype=%s",
                n_params, device, dtype)
    return model, tokenizer

In [ ]:
# ============================================================
# CELL 8 — Encoding functions with tqdm progress bars
# ============================================================

@torch.no_grad()
def encode_batch(model, tokenizer, texts: List[str], device: torch.device) -> np.ndarray:
    tokens = tokenizer(
        texts, padding=True, truncation=True,
        max_length=MAX_SEQ_LEN, return_tensors="pt",
    )
    input_ids      = tokens["input_ids"].to(device)
    attention_mask = tokens["attention_mask"].to(device)

    out    = model(input_ids=input_ids, attention_mask=attention_mask)
    hidden = out.last_hidden_state              # (B, L, H)
    mask   = attention_mask.unsqueeze(-1).float()
    pooled = (hidden * mask).sum(1) / mask.sum(1).clamp(min=1e-9)
    result = pooled.cpu().float().numpy()

    del input_ids, attention_mask, out, hidden, mask, pooled
    return result


def encode_all(model, tokenizer, texts: List[str],
               device: torch.device, batch_size: int) -> np.ndarray:
    all_emb   = []
    n         = len(texts)
    n_batches = (n + batch_size - 1) // batch_size

    pbar = tqdm(
        total=n, unit="notes", desc="Encoding with ClinicalT5",
        bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt}  [{elapsed}<{remaining}  {rate_fmt}]",
    )

    for i in range(0, n, batch_size):
        batch     = [t if t and len(t.strip()) > 0 else "patient admission"
                     for t in texts[i : i + batch_size]]
        batch_idx = i // batch_size + 1

        try:
            emb = encode_batch(model, tokenizer, batch, device)

        except RuntimeError as e:
            if "out of memory" in str(e).lower():
                logger.warning("⚠️  OOM on batch %d/%d — retrying one-by-one",
                               batch_idx, n_batches)
                if device.type == "cuda":
                    torch.cuda.empty_cache()
                gc.collect()
                emb = np.vstack([
                    encode_batch(model, tokenizer, [t], device) for t in batch
                ])
            else:
                raise

        all_emb.append(emb)
        pbar.update(len(batch))

        if device.type == "cuda":
            torch.cuda.empty_cache()
        gc.collect()

        # Periodic milestone log every 200 batches
        if batch_idx % 200 == 0 or batch_idx == n_batches:
            done = min(i + batch_size, n)
            logger.info("  [batch %d/%d]  %d/%d notes encoded  (%.1f%%)",
                        batch_idx, n_batches, done, n, done / n * 100)

    pbar.close()
    result = np.vstack(all_emb).astype(np.float32)
    logger.info("✅ Encoding complete — shape: %s", result.shape)
    return result

In [ ]:
# ============================================================
# CELL 8b — Checkpointed encode_all (resume-safe)
# ============================================================

CHECKPOINT_DIR  = os.path.join(EMBEDDINGS_DIR, "checkpoints")
CHECKPOINT_SIZE = 5000   # save every N notes

os.makedirs(CHECKPOINT_DIR, exist_ok=True)

def encode_all_checkpointed(model, tokenizer, texts, feat_hadm,
                             device, batch_size):
    """
    Encodes texts in batches, saving a .npy checkpoint every
    CHECKPOINT_SIZE notes. On restart, resumes from last checkpoint.
    """
    n         = len(texts)
    all_emb   = []
    start_idx = 0

    # ── Resume: find latest checkpoint ───────────────────────
    existing = sorted([
        f for f in os.listdir(CHECKPOINT_DIR)
        if f.startswith("emb_chunk_") and f.endswith(".npy")
    ])
    if existing:
        for fname in existing:
            chunk = np.load(os.path.join(CHECKPOINT_DIR, fname))
            all_emb.append(chunk)
        start_idx = sum(c.shape[0] for c in all_emb)
        logger.info("  ▶️  Resuming from checkpoint — %d / %d notes already done",
                    start_idx, n)
    else:
        logger.info("  Starting fresh encoding ...")

    if start_idx >= n:
        logger.info("  ✅ All notes already encoded in checkpoints")
        return np.vstack(all_emb).astype(np.float32)

    # ── Encode remaining ─────────────────────────────────────
    remaining   = texts[start_idx:]
    n_remaining = len(remaining)
    n_batches   = (n_remaining + batch_size - 1) // batch_size
    chunk_buf   = []   # accumulate until next checkpoint save

    pbar = tqdm(
        total=n_remaining, unit="notes",
        initial=0, desc="Encoding",
        bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining} {rate_fmt}]",
    )

    for i in range(0, n_remaining, batch_size):
        batch     = [t if t and len(t.strip()) > 0 else "patient admission"
                     for t in remaining[i : i + batch_size]]
        batch_idx = i // batch_size + 1

        try:
            emb = encode_batch(model, tokenizer, batch, device)
        except RuntimeError as e:
            if "out of memory" in str(e).lower():
                logger.warning("⚠️  OOM batch %d — retrying one-by-one", batch_idx)
                torch.cuda.empty_cache(); gc.collect()
                emb = np.vstack([
                    encode_batch(model, tokenizer, [t], device) for t in batch
                ])
            else:
                raise

        chunk_buf.append(emb)
        pbar.update(len(batch))

        if device.type == "cuda":
            torch.cuda.empty_cache()

        # ── Save checkpoint every CHECKPOINT_SIZE notes ───────
        done_so_far = start_idx + i + len(batch)
        if done_so_far % CHECKPOINT_SIZE < batch_size or (i + batch_size) >= n_remaining:
            if chunk_buf:
                chunk_arr  = np.vstack(chunk_buf).astype(np.float32)
                chunk_idx  = len(all_emb)
                chunk_path = os.path.join(CHECKPOINT_DIR, f"emb_chunk_{chunk_idx:04d}.npy")
                np.save(chunk_path, chunk_arr)
                all_emb.append(chunk_arr)
                chunk_buf = []
                logger.info("  💾 Checkpoint saved: %s  (%d / %d notes, %.1f%%)",
                            os.path.basename(chunk_path),
                            done_so_far, n, done_so_far / n * 100)

        gc.collect()

    pbar.close()

    # Flush any remaining buffer
    if chunk_buf:
        chunk_arr  = np.vstack(chunk_buf).astype(np.float32)
        chunk_idx  = len(all_emb)
        chunk_path = os.path.join(CHECKPOINT_DIR, f"emb_chunk_{chunk_idx:04d}.npy")
        np.save(chunk_path, chunk_arr)
        all_emb.append(chunk_arr)
        logger.info("  💾 Final chunk saved: %s", os.path.basename(chunk_path))

    result = np.vstack(all_emb).astype(np.float32)
    logger.info("✅ Encoding complete — shape: %s", result.shape)
    return result

print("✅ Checkpointed encoder defined")

✅ Checkpointed encoder defined


In [ ]:
# ============================================================
# CELL 9 — Run full pipeline  ▶️ Execute this cell to start
# ============================================================

def run():
    logger.info("=" * 60)
    logger.info("TRANCE — ClinicalT5 Embedding Pipeline")
    logger.info("=" * 60)

    # ── Step 1: Load cohort ───────────────────────────────────
    logger.info("[1/6] Loading cohort from features.csv ...")
    if not os.path.exists(FEATURES_CSV):
        raise FileNotFoundError(
            f"\n❌ features.csv not found at:\n   {FEATURES_CSV}\n"
            "Update FEATURES_CSV in Cell 3 with the correct path."
        )
    feat_df     = pd.read_csv(FEATURES_CSV, usecols=["hadm_id", "readmit_30"])
    cohort_hadm = set(feat_df["hadm_id"].astype(int))
    feat_hadm   = feat_df["hadm_id"].tolist()
    n_pos       = int(feat_df["readmit_30"].sum())
    logger.info("  Admissions : %d", len(cohort_hadm))
    logger.info("  Readmit=1  : %d (%.1f%%)", n_pos, n_pos / len(feat_df) * 100)
    logger.info("  Readmit=0  : %d (%.1f%%)", len(feat_df) - n_pos,
                (len(feat_df) - n_pos) / len(feat_df) * 100)

    # ── DEBUG MODE: set to True to test on a small subset ────────
    DEBUG        = False
    DEBUG_SIZE   = 100   # number of admissions to test on

    if DEBUG:
        import random
        random.seed(RANDOM_STATE)
        feat_hadm   = random.sample(feat_hadm, min(DEBUG_SIZE, len(feat_hadm)))
        cohort_hadm = set(feat_hadm)
        logger.info("🐛 DEBUG MODE — subset to %d admissions", len(feat_hadm))

    # ── Step 2: Load notes ───────────────────────────────────
    TEXTS_CACHE = os.path.join(PROCESSED_DIR, "preprocessed_texts.pkl")

    if os.path.exists(TEXTS_CACHE):
        logger.info("[2/6] ✅ Loading cached preprocessed texts ...")
        import pickle
        with open(TEXTS_CACHE, "rb") as f:
            cache = pickle.load(f)
        texts     = cache["texts"]
        feat_hadm = cache["feat_hadm"]
        logger.info("  %d texts loaded | avg len: %.0f chars",
                    len(texts), np.mean([len(t) for t in texts]))
    else:
        logger.info("[2/6] Indexing note files ...")
        file_index = build_file_index([MIMIC_NOTE_DIR, MIMIC_BHC_DIR])
        logger.info("  %d files indexed", len(file_index))

        logger.info("[2/6] Loading and preprocessing notes ...")
        notes        = load_notes(cohort_hadm, file_index)
        hadm_to_text = dict(zip(notes["hadm_id"], notes["text"]))
        texts        = [hadm_to_text.get(h, "patient hospitalization") for h in feat_hadm]

        n_matched  = sum(h in hadm_to_text for h in feat_hadm)
        n_fallback = len(feat_hadm) - n_matched
        logger.info("  Notes matched : %d / %d (%.1f%%)",
                    n_matched, len(feat_hadm), n_matched / len(feat_hadm) * 100)
        logger.info("  Fallback text : %d admissions", n_fallback)
        logger.info("  Avg text len  : %.0f chars", np.mean([len(t) for t in texts]))

        logger.info("  💾 Saving preprocessed texts cache ...")
        import pickle
        with open(TEXTS_CACHE, "wb") as f:
            pickle.dump({"texts": texts, "feat_hadm": feat_hadm}, f)
        logger.info("  ✅ Saved to %s (~%.0f MB)",
                    TEXTS_CACHE, os.path.getsize(TEXTS_CACHE) / 1e6)

    # ── Step 3: Device ───────────────────────────────────────
    logger.info("[3/6] Configuring compute device ...")
    device     = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    batch_size = GPU_BATCH if device.type == "cuda" else CPU_BATCH

    if device.type == "cuda":
        props = torch.cuda.get_device_properties(0)
        logger.info("  GPU       : %s", props.name)
        logger.info("  VRAM      : %.1f GB", props.total_memory / 1e9)
        logger.info("  Batch size: %d", batch_size)
    else:
        logger.warning("  ⚠️  CPU only — encoding will be very slow.")
        logger.warning("     Go to Runtime → Change runtime type → T4 GPU")

    # ── Step 4: Load model ───────────────────────────────────
    logger.info("[4/6] Loading ClinicalT5 ...")
    model, tokenizer = load_clinical_t5(device)

    # ── Step 5: Encode ───────────────────────────────────────
    logger.info("[5/6] Encoding %d texts (batch_size=%d, checkpoint every %d) ...",
                len(texts), batch_size, CHECKPOINT_SIZE)
    raw = encode_all_checkpointed(model, tokenizer, texts, feat_hadm,
                                   device, batch_size)

    zero_ratio = float((raw == 0).mean())
    var_mean   = float(raw.var(axis=0).mean())
    logger.info("  Quality — zero_ratio=%.4f | var_mean=%.6f", zero_ratio, var_mean)

    if zero_ratio > 0.9 or var_mean < 1e-7:
        raise ValueError(
            f"❌ Embedding quality too low — zero={zero_ratio:.3f}, var={var_mean:.2e}\n"
            "The model may have loaded incorrectly. Try Runtime → Restart and run all."
        )
    logger.info("  ✅ Quality check passed")

    del model
    if device.type == "cuda":
        torch.cuda.empty_cache()
    gc.collect()

    # ── Step 6: PCA + save ───────────────────────────────────
    logger.info("[6/6] PCA: %d → %d dims ...", raw.shape[1], EMBEDDING_DIM)
    scaler  = StandardScaler()
    raw_std = scaler.fit_transform(raw)
    del raw; gc.collect()

    n_comp = min(EMBEDDING_DIM, raw_std.shape[1], raw_std.shape[0] - 1)
    pca    = PCA(n_components=n_comp, random_state=RANDOM_STATE, svd_solver="randomized")

    with tqdm(total=1, desc="Fitting PCA", bar_format="{l_bar}{bar}| {elapsed}"):
        embeddings = pca.fit_transform(raw_std).astype(np.float32)

    explained = pca.explained_variance_ratio_.sum() * 100
    logger.info("  Components       : %d", n_comp)
    logger.info("  Explained variance: %.1f%%", explained)
    del raw_std; gc.collect()

    n_dims = embeddings.shape[1]
    emb_df = pd.DataFrame(embeddings, columns=[f"ct5_{i}" for i in range(n_dims)])
    emb_df["hadm_id"] = feat_hadm
    emb_df.to_csv(EMBEDDINGS_CSV, index=False)

    model_info = {
        "method" : "transformer_t5",
        "model"  : MODEL_NAME,
        "scaler" : scaler,
        "pca"    : pca,
        "dim"    : n_comp,
    }
    joblib.dump(model_info, EMBEDDING_INFO_PKL)

    logger.info("=" * 60)
    logger.info("✅ COMPLETE")
    logger.info("   Embeddings : %s", EMBEDDINGS_CSV)
    logger.info("   Pipeline   : %s", EMBEDDING_INFO_PKL)
    logger.info("   Shape      : %d admissions × %d dims", len(feat_hadm), n_dims)
    logger.info("   PCA var    : %.1f%%", explained)
    logger.info("=" * 60)


In [ ]:
run()

09:37:23  INFO      ============================================================
09:37:23  INFO      TRANCE — ClinicalT5 Embedding Pipeline
09:37:23  INFO      ============================================================
09:37:23  INFO      [1/6] Loading cohort from features.csv ...
09:37:56  INFO        Admissions : 546028
09:37:56  INFO        Readmit=1  : 101605 (18.6%)
09:37:56  INFO        Readmit=0  : 444433 (81.4%)
09:37:56  INFO      [2/6] ✅ Loading cached preprocessed texts ...
09:38:23  INFO        546038 texts loaded | avg len: 2295 chars
09:38:23  INFO      [3/6] Configuring compute device ...
09:38:23  INFO        GPU       : Tesla T4
09:38:23  INFO        VRAM      : 15.6 GB
09:38:23  INFO        Batch size: 64
09:38:23  INFO      [4/6] Loading ClinicalT5 ...
09:38:34  INFO      Loading ClinicalT5 (luqh/ClinicalT5-base) ...
09:38:35  INFO      HTTP Request: HEAD https://huggingface.co/luqh/ClinicalT5-base/resolve/main/config.json "HTTP/1.1 307 Temporary Redirect"
09:38:35

config.json:   0%|          | 0.00/664 [00:00<?, ?B/s]

09:38:35  INFO      HTTP Request: HEAD https://huggingface.co/luqh/ClinicalT5-base/resolve/main/tokenizer_config.json "HTTP/1.1 307 Temporary Redirect"
09:38:35  WARNING   Warning: You are sending unauthenticated requests to the HF Hub. Please set a HF_TOKEN to enable higher rate limits and faster downloads.
09:38:35  INFO      HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/luqh/ClinicalT5-base/f40af1cb2f1c7e257ec5974f63fd9ff224a013c9/tokenizer_config.json "HTTP/1.1 200 OK"
09:38:35  INFO      HTTP Request: GET https://huggingface.co/api/resolve-cache/models/luqh/ClinicalT5-base/f40af1cb2f1c7e257ec5974f63fd9ff224a013c9/tokenizer_config.json "HTTP/1.1 200 OK"


tokenizer_config.json: 0.00B [00:00, ?B/s]

09:38:35  INFO      HTTP Request: GET https://huggingface.co/api/models/luqh/ClinicalT5-base/tree/main/additional_chat_templates?recursive=false&expand=false "HTTP/1.1 404 Not Found"
09:38:35  INFO      HTTP Request: GET https://huggingface.co/api/models/luqh/ClinicalT5-base/tree/main?recursive=true&expand=false "HTTP/1.1 200 OK"
09:38:36  INFO      HTTP Request: HEAD https://huggingface.co/luqh/ClinicalT5-base/resolve/main/spiece.model "HTTP/1.1 302 Found"
09:38:36  INFO      HTTP Request: GET https://huggingface.co/api/models/luqh/ClinicalT5-base/xet-read-token/f40af1cb2f1c7e257ec5974f63fd9ff224a013c9 "HTTP/1.1 200 OK"


...

09:38:55  INFO        ✅ Conversion complete


Loading weights:   0%|          | 0/99 [00:00<?, ?it/s]

09:38:56  INFO      ✅ ClinicalT5 ready — 109.6M params | device=cuda | dtype=torch.float16
09:38:56  INFO      [5/6] Encoding 546038 texts (batch_size=64, checkpoint every 5000) ...
09:40:03  INFO        ▶️  Resuming from checkpoint — 546038 / 546038 notes already done
09:40:03  INFO        ✅ All notes already encoded in checkpoints
09:40:08  INFO        Quality — zero_ratio=0.0000 | var_mean=0.543198
09:40:08  INFO        ✅ Quality check passed
09:40:09  INFO      [6/6] PCA: 768 → 128 dims ...


Fitting PCA:   0%|          | 00:00

09:40:52  INFO        Components       : 128
09:40:52  INFO        Explained variance: 99.4%
09:42:14  INFO      ============================================================
09:42:14  INFO      ✅ COMPLETE
09:42:14  INFO         Embeddings : /content/drive/MyDrive/BTech-Final-Project/data/processed/embeddings/embeddings.csv
09:42:14  INFO         Pipeline   : /content/drive/MyDrive/BTech-Final-Project/data/processed/embeddings/embedding_info.pkl
09:42:14  INFO         Shape      : 546038 admissions × 128 dims
09:42:14  INFO         PCA var    : 99.4%
09:42:14  INFO      ============================================================


Checkpoint Clean UP

In [ ]:
# import shutil
# shutil.rmtree(CHECKPOINT_DIR)
# print("Checkpoints cleaned up")

In [ ]:
# # ── DIAGNOSTIC: inspect raw msgpack structure ─────────────────
# import msgpack

# msgpack_path = "/root/.cache/huggingface/hub/models--luqh--ClinicalT5-base/snapshots/f40af1cb2f1c7e257ec5974f63fd9ff224a013c9/flax_model.msgpack"

# with open(msgpack_path, "rb") as f:
#     raw = msgpack.unpackb(f.read(), raw=True, strict_map_key=False)

# # Drill into encoder → block → 0 to see what an actual leaf looks like
# def drill(obj, depth=0, max_depth=6, prefix=""):
#     if depth > max_depth:
#         return
#     if isinstance(obj, dict):
#         for i, (k, v) in enumerate(obj.items()):
#             if i > 2: break   # only show first 3 keys at each level
#             key = k.decode() if isinstance(k, bytes) else str(k)
#             print("  " * depth + f"[dict] {key}:")
#             drill(v, depth+1, max_depth, key)
#     elif isinstance(obj, list):
#         print("  " * depth + f"[list] len={len(obj)}, types={[type(x).__name__ for x in obj[:4]]}")
#         if len(obj) <= 4:
#             for i, v in enumerate(obj):
#                 drill(v, depth+1, max_depth, f"{prefix}[{i}]")
#     elif isinstance(obj, bytes):
#         print("  " * depth + f"[bytes] len={len(obj)}")
#     elif isinstance(obj, (int, float)):
#         print("  " * depth + f"[scalar] {obj}")
#     else:
#         print("  " * depth + f"[{type(obj).__name__}] {repr(obj)[:60]}")

# drill(raw)